<a href="https://colab.research.google.com/github/natthaphon-wr/cpe393-codingAI/blob/main/Lab8_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 8: Training Deep Recurrent Neural Network - Part 2
Natthaphon Rotechanathamcharoen 62070501019 <br>

**Note: Please name your file**

## Lab Instruction - Language Modelling and Text Classification

In this lab, you will learn to train a deep recurrent neural network using LSTM with the Keras library using the Tensorflow backend. Your task is to implement the natural language modelling and text generation.

Select your favourite book from https://www.gutenberg.org/browse/scores/top and download it as a text file. Then, you will train your language model using RNN-LSTM. 

- Language model (in Thai): http://bit.ly/language_model_1
- Tutorial on how to create a language model (in English): https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

To evaluate the model, the perplexity measurement is used: https://stats.stackexchange.com/questions/10302/what-is-perplexity

Last, fine-tune your model. You have to try different hyperparameter or adding more data. Discuss your result.



**The total lab score is 15 which will be evaluated as follows:**</br>
1. Specification (Do as the instruction said. This include the model tuning section where you have to do a proper amount of tuning) - 7 points
2. Design of logic (No weired things in the process) -  4points
3. Journaling (Communicate your thought process, comment your code, and discuss result & analyse **in every step**) - 4 points



###Import Library and Custom Function

In [ ]:
# Import require library
from keras import *
from keras.preprocessing import text
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

import _utils as fn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

from keras import models

In [ ]:
#from keras.layers import Embedding 
#from keras.layers import LSTM
#from keras.layers import Dropout 
#from keras.layers import Dense
from keras import layers
import keras.backend 

In [ ]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity

In [ ]:
# Create a function to evaluate your model using perplexity measurment (You can try adding other measurements as well)
def evaluate_result(features, label, model):
    model.evaluate(features, label)

### 1. Load your data 

In [ ]:
# Load data
import csv

# Load data
file = open("/content/pride_and_prejudice.txt", "r", encoding="utf-8-sig",  errors='ignore') #encode with BOM
raw_text = file.read()

In [ ]:
# See raw data
raw_text[:300]

'Chapter 1\n\n      It is a truth universally acknowledged, that a single man in\n      possession of a good fortune, must be in want of a wife.\n\n      However little known the feelings or views of such a man may be\n      on his first entering a neighbourhood, this truth is so well\n      fixed in the mi'

In [ ]:
# Get total word and character
chars = sorted(list(set(raw_text)))
print("Total characters: ", len(chars))
print("Total word: ", len(raw_text.split()))

Total characters:  89
Total word:  124467


### 2. Data Preprocessing 

*Note that only story will be used as a dataset, footnote and creddit are not include.*

The symbol '\n' is indicated the end of the line ``<EOS>``, which is for our model to end the sentence here.

To create a corpus for your model. The following code is can be used:</br>
*Note that other techniques can be used*

```python
# cut the text in semi-redundant sequences of maxlen characters.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
```

The code loop through the data from first word to the last word. The maxlen define a next n word for a model to predict.


In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
# Adding end of string symbol use .replace   to replace data_text with  [  \n\n', " <EOS> " ]
raw_text = raw_text.replace('\n\n', " <EOS> ")
raw_text[:300]

'Chapter 1 <EOS>       It is a truth universally acknowledged, that a single man in\n      possession of a good fortune, must be in want of a wife. <EOS>       However little known the feelings or views of such a man may be\n      on his first entering a neighbourhood, this truth is so well\n      fixed'

In [ ]:
# Create corpus & Vectorization

tokenizer = text.Tokenizer()

# basic cleanup
corpus = raw_text.lower().split("\n")

# tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# max sequence for padding 
max_sequence_len = max([len(x) for x in input_sequences])

# Pre padding 
#input_sequences = np.array(sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

# One-hot label
label = keras.utils.to_categorical(label, num_classes=total_words)

In [ ]:
# Find some
#print('Max sequence len: %s' % max_sequence_len)
print('Total word len: %s' % total_words)

Total word len: 7230


In [ ]:
n_gram_sequence[0]

7228

In [ ]:
print(predictors[0]) #after padding

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0 265]


In [ ]:
print(label[0])

[0. 0. 0. ... 0. 0. 0.]


### 3. Language Model

Define RNN model using LSTM and word embedding representation</br>
We will used perplexity as a metrics

```python
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```

To used custom metrics function > https://keras.io/metrics/

For a loss function `categorical_crossentropy` is used, any optimzation method can be applied.


In this lab, we will used perplexity as a metrics


```
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```


To used custom metrics function > https://keras.io/metrics/

In [ ]:
# Define your model

# LSTM Layer use DEFAULT weight initializer = glorot/xavier uniform initializer,
#            add dropout for avoid overfitting
# If Use more than 1 LSTM layer must setting return_sequebce = True
# Try using perplexity metrics

model = models.Sequential()
model.add(layers.Embedding(total_words, 128, input_length = max_sequence_len-1, name='Embedding'))
model.add(layers.LSTM(128, dropout = 0.2, name = 'LSTM1'))
model.add(layers.Dense(total_words, activation = 'softmax', name = 'Output'))

model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = [perplexity])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 60, 128)           925440    
                                                                 
 LSTM1 (LSTM)                (None, 128)               131584    
                                                                 
 Output (Dense)              (None, 7230)              932670    
                                                                 
Total params: 1,989,694
Trainable params: 1,989,694
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit Model
history = model.fit(predictors, label, batch_size=128, epochs=10)
model.save('model_baseline.h5')

Epoch 1/10
930/930 [==============================] - 16s 10ms/step - loss: 6.2994 - perplexity: 654.5432
Epoch 2/10
930/930 [==============================] - 10s 10ms/step - loss: 5.7359 - perplexity: 310.3341
Epoch 3/10
930/930 [==============================] - 10s 11ms/step - loss: 5.3554 - perplexity: 237.2975
Epoch 4/10
930/930 [==============================] - 10s 11ms/step - loss: 5.1197 - perplexity: 183.0687
Epoch 5/10
930/930 [==============================] - 10s 10ms/step - loss: 4.9360 - perplexity: 147.6357
Epoch 6/10
930/930 [==============================] - 10s 11ms/step - loss: 4.7835 - perplexity: 120.8282
Epoch 7/10
930/930 [==============================] - 10s 10ms/step - loss: 4.6505 - perplexity: 101.1184
Epoch 8/10
930/930 [==============================] - 9s 10ms/step - loss: 4.5345 - perplexity: 86.5859
Epoch 9/10
930/930 [==============================] - 10s 11ms/step - loss: 4.4250 - perplexity: 76.3589
Epoch 10/10
930/930 [============================

### 4. Evaluate your model 

In [ ]:
# Evaluate Model
model = models.load_model('/content/model_baseline.h5', custom_objects = {'perplexity': perplexity})
evaluate_result(predictors, label, model)

3720/3720 [==============================] - 24s 5ms/step - loss: 4.1282 - perplexity: 52.4014


###5. First Discussion and This Experiment

In summary of my understanding about perplexity, perplexity is a metric about probability of words and entropy, and it's the better to evaluate than loss/accuracy. The lower perplexity refers to better language model. According to the first model "model_baseline", evaluated perplexity is 52.4014. It's quite low.

In this experiment, I will use perplexity to evaluate models, and use tuning some hyperparameter to find the different results.

###Tuning Hyperparameter

####Add Second LSTM Layer
My assumption is that model is better (perplexity is lower).

In [ ]:
# Define your model

# LSTM Layer use DEFAULT weight initializer = glorot/xavier uniform initializer,
#            add dropout for avoid overfitting
# If Use more than 1 LSTM layer must setting return_sequences = True

model2 = models.Sequential()
model2.add(layers.Embedding(total_words, 128, input_length = max_sequence_len-1, name='Embedding'))
model2.add(layers.LSTM(128, dropout = 0.2, return_sequences = True, name = 'LSTM1'))
model2.add(layers.LSTM(128, dropout = 0.2, name = 'LSTM2'))
model2.add(layers.Dense(total_words, activation = 'softmax', name = 'Output'))

model2.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = [perplexity])
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 60, 128)           925440    
                                                                 
 LSTM1 (LSTM)                (None, 60, 128)           131584    
                                                                 
 LSTM2 (LSTM)                (None, 128)               131584    
                                                                 
 Output (Dense)              (None, 7230)              932670    
                                                                 
Total params: 2,121,278
Trainable params: 2,121,278
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit Model
history = model2.fit(predictors, label, batch_size=128, epochs=10)
model2.save('model_2LSTM.h5')

In [ ]:
# Evaluate Model
model = models.load_model('/content/model_2LSTM.h5', custom_objects = {'perplexity': perplexity})
evaluate_result(predictors, label, model)

3720/3720 [==============================] - 30s 6ms/step - loss: 4.5668 - perplexity: 71.4377


The perplexity of "model_2LSTM" is 71.4377 (more than 1 layer LSTM). It's not result that I expected. After research a few simple language models with LSTM, there is no model that have more than 1 LSTM layer. I think it's not necessary process(it's not matter) or over-processing. 

####No Dropout 
This tuning is rather sure to improve the model, but let's see.

In [ ]:
# Define your model
# LSTM Layer use DEFAULT weight initializer = glorot/xavier uniform initializer,

model3 = models.Sequential()
model3.add(layers.Embedding(total_words, 128, input_length = max_sequence_len-1, name='Embedding'))
model3.add(layers.LSTM(128, name = 'LSTM1'))
model3.add(layers.Dense(total_words, activation = 'softmax', name = 'Output'))

model3.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = [perplexity])
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 60, 128)           925440    
                                                                 
 LSTM1 (LSTM)                (None, 128)               131584    
                                                                 
 Output (Dense)              (None, 7230)              932670    
                                                                 
Total params: 1,989,694
Trainable params: 1,989,694
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit Model
history = model3.fit(predictors, label, batch_size=128, epochs=10)
model3.save('model_noDrop.h5')

Epoch 1/10
930/930 [==============================] - 16s 11ms/step - loss: 6.2934 - perplexity: 649.1934
Epoch 2/10
930/930 [==============================] - 10s 11ms/step - loss: 5.6716 - perplexity: 299.9314
Epoch 3/10
930/930 [==============================] - 10s 10ms/step - loss: 5.3027 - perplexity: 227.7965
Epoch 4/10
930/930 [==============================] - 10s 10ms/step - loss: 5.0756 - perplexity: 176.0975
Epoch 5/10
930/930 [==============================] - 10s 10ms/step - loss: 4.8942 - perplexity: 141.2868
Epoch 6/10
930/930 [==============================] - 10s 10ms/step - loss: 4.7400 - perplexity: 115.4907
Epoch 7/10
930/930 [==============================] - 10s 10ms/step - loss: 4.6025 - perplexity: 96.6701
Epoch 8/10
930/930 [==============================] - 10s 10ms/step - loss: 4.4775 - perplexity: 82.2302
Epoch 9/10
930/930 [==============================] - 9s 10ms/step - loss: 4.3611 - perplexity: 71.7442
Epoch 10/10
930/930 [=============================

In [ ]:
# Evaluate Model
model = models.load_model('/content/model_noDrop.h5', custom_objects = {'perplexity': perplexity})
evaluate_result(predictors, label, model)

3720/3720 [==============================] - 23s 5ms/step - loss: 4.0681 - perplexity: 51.0891


The perplexity of "model_2LSTM" is 51.0891. It's a little less than the baseline model, and maybe not significant. 

Now, I'm quite confident that this architecture (embedding, 1 layer of LSTM, and Dense for output) is fine. There is still lots of hyperparameter that can tune, but I think tuning regularizer is not help. Moreover, there is no massive specific reason that to tune initializer, constraint, and optimizer. 

In the next experiment, I will try increase number of LSTM units and decrease batch size to see how much significant change the perplexity.

####Increase LSTM Units

In [ ]:
# Define your model
# LSTM Layer use DEFAULT weight initializer = glorot/xavier uniform initializer,
# No dropout
# Increase LSTM units x2 (128 -> 256)
# If increase LSTM -> must increase output dimension of embedding layer too

model4 = models.Sequential()
model4.add(layers.Embedding(total_words, 256, input_length = max_sequence_len-1, name='Embedding'))
model4.add(layers.LSTM(256, name = 'LSTM'))
model4.add(layers.Dense(total_words, activation = 'softmax', name = 'Output'))

model4.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = [perplexity])
model4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 60, 256)           1850880   
                                                                 
 LSTM (LSTM)                 (None, 256)               525312    
                                                                 
 Output (Dense)              (None, 7230)              1858110   
                                                                 
Total params: 4,234,302
Trainable params: 4,234,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit Model
history = model4.fit(predictors, label, batch_size=128, epochs=10)
model4.save('model_increUnit.h5')

Epoch 1/10
930/930 [==============================] - 20s 15ms/step - loss: 6.1288 - perplexity: 685.9318
Epoch 2/10
930/930 [==============================] - 13s 14ms/step - loss: 5.3582 - perplexity: 300.4081
Epoch 3/10
930/930 [==============================] - 13s 14ms/step - loss: 4.9892 - perplexity: 179.6489
Epoch 4/10
930/930 [==============================] - 13s 14ms/step - loss: 4.7143 - perplexity: 119.7724
Epoch 5/10
930/930 [==============================] - 13s 14ms/step - loss: 4.4826 - perplexity: 85.9898
Epoch 6/10
930/930 [==============================] - 14s 15ms/step - loss: 4.2701 - perplexity: 65.9632
Epoch 7/10
930/930 [==============================] - 15s 16ms/step - loss: 4.0687 - perplexity: 53.0690
Epoch 8/10
930/930 [==============================] - 14s 15ms/step - loss: 3.8719 - perplexity: 43.8041
Epoch 9/10
930/930 [==============================] - 13s 14ms/step - loss: 3.6844 - perplexity: 37.5160
Epoch 10/10
930/930 [==============================

In [ ]:
# Evaluate Model
model = models.load_model('/content/model_increUnit.h5', custom_objects = {'perplexity': perplexity})
evaluate_result(predictors, label, model)

3720/3720 [==============================] - 23s 5ms/step - loss: 3.2083 - perplexity: 25.0206


After increasing LSTM units 2 times, total parameters and size of model are almost double too. Perplexity of this new model is 25.0206. It's dramatically improve.

####Decrease Batch Size

In [ ]:
# Define your model
# LSTM Layer use DEFAULT weight initializer = glorot/xavier uniform initializer,
# No dropout
# Increase LSTM units x2 (128 -> 256)
# If increase LSTM -> must increase output dimension of embedding layer too

model5 = models.Sequential()
model5.add(layers.Embedding(total_words, 256, input_length = max_sequence_len-1, name='Embedding'))
model5.add(layers.LSTM(256, name = 'LSTM'))
model5.add(layers.Dense(total_words, activation = 'softmax', name = 'Output'))

model5.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = [perplexity])
model5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 60, 256)           1850880   
                                                                 
 LSTM (LSTM)                 (None, 256)               525312    
                                                                 
 Output (Dense)              (None, 7230)              1858110   
                                                                 
Total params: 4,234,302
Trainable params: 4,234,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit Model
# Reduce batch size from 128 to 64
history = model5.fit(predictors, label, batch_size=64, epochs=10)
model5.save('model_increUnit_reduSize.h5')

Epoch 1/10
1860/1860 [==============================] - 25s 10ms/step - loss: 5.9658 - perplexity: 697.8634
Epoch 2/10
1860/1860 [==============================] - 23s 12ms/step - loss: 5.1481 - perplexity: 260.9930
Epoch 3/10
1860/1860 [==============================] - 20s 11ms/step - loss: 4.7593 - perplexity: 136.7756
Epoch 4/10
1860/1860 [==============================] - 19s 10ms/step - loss: 4.4358 - perplexity: 84.3082
Epoch 5/10
1860/1860 [==============================] - 19s 10ms/step - loss: 4.1291 - perplexity: 57.5445
Epoch 6/10
1860/1860 [==============================] - 19s 10ms/step - loss: 3.8307 - perplexity: 42.9745
Epoch 7/10
1860/1860 [==============================] - 19s 10ms/step - loss: 3.5429 - perplexity: 33.5638
Epoch 8/10
1860/1860 [==============================] - 19s 10ms/step - loss: 3.2707 - perplexity: 27.5337
Epoch 9/10
1860/1860 [==============================] - 19s 10ms/step - loss: 3.0151 - perplexity: 23.2480
Epoch 10/10
1860/1860 [===========

In [ ]:
# Evaluate Model
model = models.load_model('/content/model_increUnit_reduSize.h5', custom_objects = {'perplexity': perplexity})
evaluate_result(predictors, label, model)

3720/3720 [==============================] - 23s 5ms/step - loss: 2.3694 - perplexity: 13.9848


Perplexity is 13.9848. It's reduced almost 2 times following my assumption. Then, I will use this model to text generating in the next section.

### 6. Text generating

In [ ]:
def generate_text(seedtext, next_words, max_sequence_len, model):
  for j in range(next_words):
    token_list = tokenizer.texts_to_sequences([seedtext])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #predicted = model.predict_classes(token_list, verbose=0)
    predict_x = model.predict(token_list) 
    predicted = np.argmax(predict_x,axis=1)

    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seedtext +=" " + output_word
  return seedtext

In [ ]:
# Load model
model = models.load_model('/content/model_increUnit_reduSize.h5', custom_objects = {'perplexity': perplexity})

In [ ]:
# Generate your sample text
seed_text = input('Enter your start sentence:')
gen_text = generate_text(seed_text, 10, max_sequence_len, model)
gen_text

Enter your start sentence:We are
1/1 [==============================] - 0s 19ms/step


'We are very glad to be done for my own and pray'

In [ ]:
# Generate your sample text
seed_text = input('Enter your start sentence:')
gen_text = generate_text(seed_text, 10, max_sequence_len, model)
gen_text

Enter your start sentence:She can
1/1 [==============================] - 0s 17ms/step


'She can herself be the only probable person for the first object'

In [ ]:
# Generate your sample text
seed_text = input('Enter your start sentence:')
gen_text = generate_text(seed_text, 20, max_sequence_len, model)
gen_text

Enter your start sentence:You should
1/1 [==============================] - 0s 20ms/step


'You should not have a more promising for him ” eos elizabeth was then sorry for her and the two youngest repaired'

In [ ]:
# Generate your sample text
seed_text = input('Enter your start sentence:')
gen_text = generate_text(seed_text, 15, max_sequence_len, model)
gen_text

Enter your start sentence:Marriage is
1/1 [==============================] - 0s 18ms/step


'Marriage is a very good match for a young man who has been very much in love'

#####Discussion
After try some sample, sentences that are generated from given words are reasonable. We can see the meaning of them. Pride&Prejudice is a legendary novel that are uniqly portrayal. It has been praised about charming characters and usage of language. The results are showed portrayal too.

###[Speacial] 7. Help your model to generate a short story 

**Example** https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6

Write your result in a `markdown` cell

In [ ]:
# Create your short-story from your model (Add your creativity here)

### More on Natural language Processing and Language model
1. https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e 
2. https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b
3. http://karpathy.github.io/2015/05/21/rnn-effectiveness/

**Music generates by RNN**
https://soundcloud.com/optometrist-prime/recurrence-music-written-by-a-recurrent-neural-network
